## See [Keras: Sequence to sequence](https://keras.io/examples/lstm_seq2seq/)
## See [Day 18: 機器翻譯(Machine Translation) - iT 邦幫忙](https://ithelp.ithome.com.tw/articles/10194403)

In [1]:
%env KERAS_BACKEND=tensorflow
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

env: KERAS_BACKEND=tensorflow


Using TensorFlow backend.


In [2]:
batch_size = 32  # Batch size for training.
epochs = 2  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space. (i.e. there are latent_dim cells in LSTM layer)
num_samples = 10000  # Number of samples to train on. # total 21007 samples
# Path to the data txt file on disk.
data_path = 'json_h5/S2S.txt'

In [3]:
# Vectorize the data.
input_texts = [] # list of input sentences
target_texts = [] # list of output sentences
input_characters = set()
target_characters = set()
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [4]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 73
Number of unique output tokens: 2588
Max sequence length for inputs: 30
Max sequence length for outputs: 22


In [5]:
# char to int
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

#---------------------seems like one hot encoding---------------------------
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
#---------------------seems like one hot encoding---------------------------

# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

# Run training
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 73)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2588)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 337920      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  2913280     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [7]:
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, 
          batch_size=batch_size, 
          epochs=epochs, 
          shuffle=True, 
          validation_split=0.2
         )

# Save model
# model.save('json_h5/RNN_translate.h5')
model_json = model.to_json()
open('json_h5/RNN_translate.json','w').write(model_json)
model.save_weights('json_h5/RNN_translate.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/2
8000/8000 [==============================] - 113s 14ms/step - loss: 1.9973 - acc: 0.0717 - val_loss: 2.4695 - val_acc: 0.0757
Epoch 2/2
8000/8000 [==============================] - 85s 11ms/step - loss: 1.8122 - acc: 0.0996 - val_loss: 2.3363 - val_acc: 0.1107


c:\users\奎佑\appdata\local\programs\python\python36\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [10]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index + int(num_samples * 0.8): seq_index + int(num_samples * 0.8) + 1]
    decoded_sentence = decode_sequence(input_seq)
    # print('-')
    print('\nInput sentence:', input_texts[seq_index + int(num_samples * 0.8)])
    print('Decoded sentence:', decoded_sentence)


Input sentence: He drives a car, doesn't he?
Decoded sentence: 仍胃洞嫁嫁奖奖倔瓦赶巾洽钥险详東谦碰難本逻逻笑

Input sentence: He flew a kite with his son.
Decoded sentence: 逐D齊悉  溺溺嚮嚮壁f纯e满科傳蠅材甜鋪管矮

Input sentence: He got a loan from the bank.
Decoded sentence: 不海版练鄰睛砸寸摩群脅辭辭抱惡至避避場姆肉碰b

Input sentence: He got engaged to my cousin.
Decoded sentence: 不兽面萝萝她未未飲搬悄療植解迷迷丑胃乞胃镜蘋钱

Input sentence: He had barely enough to eat.
Decoded sentence: 则则許許空s沏購气刪觀繁齦齦員漉观哭挖早假沮沮

Input sentence: He hadn't eaten in two days.
Decoded sentence: 则鐘三三划氧便肥临肥参木衣花劣p說挽斷斷汤饥段

Input sentence: He has been sick for a week.
Decoded sentence: 则三三资陰什帆帆汗菠圣衣酬酬約性醇别麗麗均麗銀

Input sentence: He has more books than I do.
Decoded sentence: 则鐘三三訝划哇悉划件欢笔獄脆貌貌市越府俊物鄙預

Input sentence: He has no reason to be late.
Decoded sentence: 则则許許空s沏購气刪觀繁齦齦員漉观哭挖早假沮沮

Input sentence: He helped me do my homework.
Decoded sentence: 则三三资陰什帆帆汗菠圣衣酬約敵杆拼免不火顺顺瞭

Input sentence: He is a bit like his father.
Decoded sentence: 则三三香贵划钱轨唬唬制巾帆帆汗蜂絲鸟哇它块及佬

Input sentence: He is a head ta

# ------------------test down----------------

# ------------------test down----------------

# ------------------test down----------------

# ------------------test down----------------

In [9]:
model.fit([encoder_input_data, decoder_input_data], 
          decoder_target_data, 
          batch_size=batch_size, 
          epochs=3, 
          shuffle=True, 
          validation_split=0.2
         )
# Save model
# model.save('json_h5/RNN_translate.h5')
model_json = model.to_json()
open('json_h5/S2S.json','w').write(model_json)
model.save_weights('json_h5/S2S.h5')

Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 8000 samples, validate on 2000 samples
Epoch 1/3
8000/8000 [==============================] - 156s 19ms/step - loss: 1.2769 - acc: 0.1646 - val_loss: 1.8962 - val_acc: 0.1724
Epoch 2/3
8000/8000 [==============================] - 144s 18ms/step - loss: 1.2120 - acc: 0.1730 - val_loss: 1.8684 - val_acc: 0.1761
Epoch 3/3
8000/8000 [==============================] - 145s 18ms/step - loss: 1.1646 - acc: 0.1777 - val_loss: 1.8594 - val_acc: 0.1779


c:\users\奎佑\appdata\local\programs\python\python36\lib\site-packages\keras\engine\network.py:877: UserWarning: Layer lstm_2 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_1_1/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_1_1/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


In [10]:
from keras.models import model_from_json

new_model=model_from_json(open('json_h5/S2S.json').read())
new_model.load_weights('json_h5/S2S.h5')
new_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 73)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2588)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 337920      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  2913280     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [6]:
from keras.models import model_from_json

model=model_from_json(open('json_h5/S2S.json').read())
model.load_weights('json_h5/S2S.h5')
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, 73)     0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, None, 2588)   0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 337920      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  2913280     input_2[0][0]                    
                                                                 lstm_1[0][1]                     
          